In [4]:
#auto reload 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
from datasets import load_dataset

# Load the WMT14 dataset
wmt14 = load_dataset("wmt14", "fr-en", data_dir="data/")

# Accessing example data
train_data = wmt14["train"]
val_data = wmt14["validation"]

# Print an example from the training set
print(train_data[0])


Extracting data files: 100%|██████████| 7/7 [00:00<00:00, 3440.77it/s]
Extracting data files: 0it [00:00, ?it/s]
Generating train split:   0%|          | 183755/40836715 [00:02<10:02, 67517.88 examples/s]


KeyboardInterrupt: 